# 🚀 Fine-tune Falcon-1B on SQL Queries with LoRA (Colab Friendly)

In [ ]:

!pip install -q transformers datasets peft accelerate bitsandbytes wandb


In [ ]:
from datasets import Dataset

# ✅ Synthetic Spider-style dataset (for demo)
examples = [
    {"question": "List all employees in the sales department", "query": "SELECT * FROM employees WHERE department = 'sales'"},
    {"question": "Get names of employees earning more than 50000", "query": "SELECT name FROM employees WHERE salary > 50000"},
    {"question": "Count products sold in January", "query": "SELECT COUNT(*) FROM sales WHERE month = 'January'"},
    {"question": "Show all product names", "query": "SELECT name FROM products"},
    {"question": "Get departments with more than 5 employees", "query": "SELECT department FROM employees GROUP BY department HAVING COUNT(*) > 5"},
    {"question": "Find highest paid employee", "query": "SELECT name FROM employees ORDER BY salary DESC LIMIT 1"},
    {"question": "List employees and their managers", "query": "SELECT e.name, m.name FROM employees e JOIN employees m ON e.manager_id = m.id"},
    {"question": "Get average salary by department", "query": "SELECT department, AVG(salary) FROM employees GROUP BY department"},
    {"question": "Retrieve orders placed in last 7 days", "query": "SELECT * FROM orders WHERE date > DATE_SUB(CURDATE(), INTERVAL 7 DAY)"},
    {"question": "Total revenue from product A", "query": "SELECT SUM(price) FROM sales WHERE product = 'A'"}
]

dataset = Dataset.from_list([{
    "prompt": f"""### Question:
{item['question']}
### SQL:
""",
    "completion": item["query"]
} for item in examples])

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:

def tokenize(example):
    return tokenizer(example["prompt"] + example["completion"], truncation=True, padding="max_length", max_length=256)

tokenized = dataset.map(tokenize)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:

from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./falcon-sql",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


/tmp/ipython-input-7-1312184743.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


TrainOutput(global_step=15, training_loss=3.056664530436198, metrics={'train_runtime': 14.7456, 'train_samples_per_second': 2.035, 'train_steps_per_second': 1.017, 'total_flos': 55764886487040.0, 'train_loss': 3.056664530436198, 'epoch': 3.0})


### ✅ Once training finishes, you can push to Hugging Face using:
```python
model.push_to_hub('your-username/falcon-sql-lora')
```

In [ ]:
from huggingface_hub import login

login()  # Paste the token here when asked



In [ ]:
model.push_to_hub("revanthkumarg/falcon-sql-lora", commit_message="Finetuned Falcon SQL model")
tokenizer.push_to_hub("revanthkumarg/falcon-sql-lora")


README.md: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/revanthkumarg/falcon-sql-lora/commit/14c52c742f7f1ebf7f9b0f117e8f5709cfe37fc6', commit_message='Upload tokenizer', commit_description='', oid='14c52c742f7f1ebf7f9b0f117e8f5709cfe37fc6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/revanthkumarg/falcon-sql-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='revanthkumarg/falcon-sql-lora'), pr_revision=None, pr_num=None)

## 🧠 Build SQL RAG System (Next Phase)
- Load schema/tables
- Embed using sentence-transformers
- Store in FAISS/Chroma
- Use LangGraph to retrieve context + user query → pass to model

## 🧠 Build SQL RAG System (LangGraph + Falcon)

In [ ]:

# Install dependencies
!pip install -q langchain langgraph chromadb sentence-transformers faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 109.5 MB/s eta 

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset

def build_sql_prompt(schema_context, user_question, few_shot_examples):
    prompt = "### Schema Info:\n" + "\n".join([doc.page_content for doc in schema_context])
    prompt += "\n\n"  # Add a blank line between schema and few-shot examples
    for example in few_shot_examples:
        prompt += f"""### Question:
{example['question']}
### SQL:
{example['query']}

"""
    prompt += f"""### Question:
{user_question}
### SQL:
"""
    return prompt

# Assuming 'context' from cell 0fa5d16e contains the schema context
# schema_context = context # Comment out the line causing the error

# Provide a dummy schema_context for demonstration if the vector store is not ready
from langchain_core.documents import Document
schema_context = [
    Document(page_content='Table: employees | Columns: id, name, department, salary, manager_id', metadata={}),
    Document(page_content='Table: orders | Columns: id, product, user_id, date, amount', metadata={})
]

# ✅ Synthetic Spider-style dataset (for demo)
examples = [
    {"question": "List all employees in the sales department", "query": "SELECT * FROM employees WHERE department = 'sales'"},
    {"question": "Get names of employees earning more than 50000", "query": "SELECT name FROM employees WHERE salary > 50000"},
    {"question": "Count products sold in January", "query": "SELECT COUNT(*) FROM sales WHERE month = 'January'"},
    {"question": "Show all product names", "query": "SELECT name FROM products"},
    {"question": "Get departments with more than 5 employees", "query": "SELECT department FROM employees GROUP BY department HAVING COUNT(*) > 5"},
    {"question": "Find highest paid employee", "query": "SELECT name FROM employees ORDER BY salary DESC LIMIT 1"},
    {"question": "List employees and their managers", "query": "SELECT e.name, m.name FROM employees e JOIN employees m ON e.manager_id = m.id"},
    {"question": "Get average salary by department", "query": "SELECT department, AVG(salary) FROM employees GROUP BY department"},
    {"question": "Retrieve orders placed in last 7 days", "query": "SELECT * FROM orders WHERE date > DATE_SUB(CURDATE(), INTERVAL 7 DAY)"},
    {"question": "Total revenue from product A", "query": "SELECT SUM(price) FROM sales WHERE product = 'A'"}
]


# Assuming 'examples' from cell 85970711 contains the few-shot examples
few_shot_examples = examples

In [ ]:
!pip install -q faiss-cpu

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.documents import Document

# Initialize the embedding model
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create and populate the vectorstore (using the dummy schema_context for now)
# In a real application, you would load your actual schema information here
vectorstore = FAISS.from_documents(schema_context, embeddings)

# Now you can run the selected cell (0fa5d16e)

/tmp/ipython-input-14-1631630932.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

# Simulate user query and retrieve schema context
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

user_query = "List all employees earning more than 50000"
context = retriever.get_relevant_documents(user_query)

# Combine context and query
rag_prompt = "### Schema Info:\n" + "\n".join([doc.page_content for doc in context])
rag_prompt += f"\n### Question:\n{user_query}\n### SQL:\n"
print(rag_prompt)


/tmp/ipython-input-15-3318023928.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(user_query)


### Schema Info:
Table: employees | Columns: id, name, department, salary, manager_id
Table: orders | Columns: id, product, user_id, date, amount
### Question:
List all employees earning more than 50000
### SQL:



In [ ]:
rag_prompt = """### Schema Info:
employees(id, name, department, salary, manager_id)
orders(id, product, user_id, date, amount)

### Question:
List all employees in the IT department
### SQL:
SELECT * FROM employees WHERE department = 'IT';

### Question:
List all employees earning more than 50000
### SQL:
SELECT * FROM employees WHERE salary > 50000;

### Question:
List all employees earning less than 50000
### SQL:
SELECT * FROM employees WHERE salary < 50000;

### Question:
List all employees with a manager_id of 1
### SQL:
SELECT * FROM employees WHERE manager_id = 1;

### Question:
List all employees with a manager_id of 2
### SQL:
"""



In [ ]:
def generate_sql(user_question):
    prompt = build_sql_prompt(schema_context, user_question, few_shot_examples)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150, # Increased max_new_tokens
        pad_token_id=tokenizer.eos_token_id
    )
    # Decode and extract only the generated SQL part
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    sql_start_index = generated_text.find("### SQL:")
    if sql_start_index != -1:
        # Find the next "###" or end of string to extract the SQL
        sql_end_index = generated_text.find("###", sql_start_index + len("### SQL:"))
        if sql_end_index != -1:
            return generated_text[sql_start_index + len("### SQL:") : sql_end_index].strip()
        else:
            return generated_text[sql_start_index + len("### SQL:"):].strip()
    return "Could not generate SQL." # Return an informative message if SQL tag is not found

In [ ]:

def build_sql_prompt(schema_context, user_question, few_shot_examples):
    prompt = "### Schema Info:\n" + "\n".join([doc.page_content for doc in schema_context])
    prompt += "\n\n"  # Add a blank line between schema and few-shot examples
    for example in few_shot_examples:
        prompt += f"""### Question:
{example['question']}
### SQL:
{example['query']}

"""
    prompt += f"""### Question:
{user_question}
### SQL:
"""
    return prompt

# Assuming 'context' from cell 0fa5d16e contains the schema context
schema_context = context

# Assuming 'examples' from cell 85970711 contains the few-shot examples
few_shot_examples = examples

In [ ]:
user_query = "Show all employees whose manager is in HR"
prompt = build_sql_prompt(schema_context, user_query, few_shot_examples)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Schema Info:
Table: employees | Columns: id, name, department, salary, manager_id
Table: orders | Columns: id, product, user_id, date, amount

### Question:
List all employees in the sales department
### SQL:
SELECT * FROM employees WHERE department ='sales'

### Question:
Get names of employees earning more than 50000
### SQL:
SELECT name FROM employees WHERE salary > 50000

### Question:
Count products sold in January
### SQL:
SELECT COUNT(*) FROM sales WHERE month = 'January'

### Question:
Show all product names
### SQL:
SELECT name FROM products

### Question:
Get departments with more than 5 employees
### SQL:
SELECT department FROM employees GROUP BY department HAVING COUNT(*) > 5

### Question:
Find highest paid employee
### SQL:
SELECT name FROM employees ORDER BY salary DESC LIMIT 1

### Question:
List employees and their managers
### SQL:
SELECT e.name, m.name FROM employees e JOIN employees m ON e.manager_id = m.id

### Question:
Get average salary by department
### SQL

In [ ]:
!pip install -q streamlit transformers peft accelerate
!pip install -q pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install gradio

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load model + tokenizer
model_id = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_id)
lora_model_id = "revanthkumarg/falcon-sql-lora"
model = PeftModel.from_pretrained(base_model, lora_model_id)
model.eval()

# SQL generation function
def generate_sql(user_query):
    schema_context = """employees(id, name, department, salary, manager_id)
orders(id, product, user_id, date, amount)"""

    few_shot_examples = """
### Question:
List all employees in the IT department
### SQL:
SELECT * FROM employees WHERE department = 'IT';

### Question:
List all employees earning more than 50000
### SQL:
SELECT * FROM employees WHERE salary > 50000;
"""

    prompt = f"### Schema Info:\n{schema_context}\n\n{few_shot_examples}\n### Question:\n{user_query}\n### SQL:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("### SQL:")[-1].strip()

# Launch Gradio app
gr.Interface(fn=generate_sql,
             inputs="text",
             outputs="text",
             title="Falcon SQL Generator",
             description="Ask a question based on the schema and get SQL.").launch()


In [ ]:
generated_sql = generate_sql(rag_prompt)
print(generated_sql)

### Schema Info:
Table: employees | Columns: id, name, department, salary, manager_id
Table: orders | Columns: id, product, user_id, date, amount

### Question:
List all employees in the sales department
### SQL:
SELECT * FROM employees WHERE department ='sales'

### Question:
Get names of employees earning more than 50000
### SQL:
SELECT name FROM employees WHERE salary > 50000

### Question:
Count products sold in January
### SQL:
SELECT COUNT(*) FROM sales WHERE month = 'January'

### Question:
Show all product names
### SQL:
SELECT name FROM products

### Question:
Get departments with more than 5 employees
### SQL:
SELECT department FROM employees GROUP BY department HAVING COUNT(*) > 5

### Question:
Find highest paid employee
### SQL:
SELECT name FROM employees ORDER BY salary DESC LIMIT 1

### Question:
List employees and their managers
### SQL:
SELECT e.name, m.name FROM employees e JOIN employees m ON e.manager_id = m.id

### Question:
Get average salary by department
### SQL

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

BASE_MODEL = "tiiuae/falcon-rw-1b"
LORA_MODEL = "revanthkumarg/falcon-sql-lora"

@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, trust_remote_code=True)
    model = PeftModel.from_pretrained(base_model, LORA_MODEL)
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()

st.title("🧠 SQL Generator using Falcon + LoRA")

schema_input = st.text_area("📑 Schema (Example: employees(id, name, ...))",
"""
employees(id, name, department, salary, manager_id)
orders(id, product, user_id, date, amount)
""")

user_question = st.text_input("❓ Ask a Question (e.g., List all employees in IT)")

if st.button("🚀 Generate SQL") and user_question:
    prompt = f"""### Schema Info:\n{schema_input}\n\n### Question:\n{user_question}\n### SQL:"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    # Increased max_new_tokens for potentially longer SQL queries
    outputs = model.generate(**inputs, max_new_tokens=500)
    sql = tokenizer.decode(outputs[0], skip_special_tokens=True).split("### SQL:")[-1].strip()
    st.code(sql, language="sql")

Overwriting app.py


In [ ]:


import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load model + tokenizer
model_id = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_id)
lora_model_id = "revanthkumarg/falcon-sql-lora"
model = PeftModel.from_pretrained(base_model, lora_model_id)
model.eval()

# SQL generation function
def generate_sql(user_query):
    schema_context = """employees(id, name, department, salary, manager_id)
orders(id, product, user_id, date, amount)"""

    few_shot_examples = """
### Question:
List all employees in the IT department
### SQL:
SELECT * FROM employees WHERE department = 'IT';

### Question:
List all employees earning more than 50000
### SQL:
SELECT * FROM employees WHERE salary > 50000;
"""

    prompt = f"### Schema Info:\n{schema_context}\n\n{few_shot_examples}\n### Question:\n{user_query}\n### SQL:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=200)

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("### SQL:")[-1].strip()

# Launch Gradio app
gr.Interface(fn=generate_sql,
             inputs="text",
             outputs="text",
             title="Falcon SQL Generator",
             description="Ask a question based on the schema and get SQL.").launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://efbbf44becad4a32f7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
